# Monads
https://www.pandaandthegeek.com/post/monads-scala

In [1]:
static int? DivBy(int a, int b) =>
    b == 0 ? default : a / b;

static int? Sample_Nullable() {
    var a = 1;
    var b = 2;

    var c = DivBy(a, b);
    if (c is null){
        Console.WriteLine("c is null");
        return default;
    }

    var d = DivBy(a, c.Value);
    if (d is null){
        Console.WriteLine("d is null");
        return default;
    }

    var e = DivBy(b, d.Value);
    if (e is null) {
        Console.WriteLine("e is null");
        return default;
    }
        

    return e;
}

Console.WriteLine(Sample_Nullable())

0


In [ ]:
public enum StatusCode {
    Ok,
    DivByZero
}

static (StatusCode, int) DivBy(int a, int b) =>
    b == 0 ? (StatusCode.DivByZero, default) : (StatusCode.Ok, a / b);

static (StatusCode, int) Sample_StatusCode() {
    var a = 1;
    var b = 2;

    var (statusC, c) = DivBy(a, b);
    
    if (statusC is StatusCode.DivByZero){
        Console.WriteLine($"e did not work {statusC}");
        return (StatusCode.DivByZero, default);    
    }

    var (statusD, d) = DivBy(a, c);
    if (statusD is StatusCode.DivByZero) {
        Console.WriteLine($"e did not work {statusD}");
        return (StatusCode.DivByZero, default);    
    }

    var (statusE, e) = DivBy(a, c);
    if (statusE is StatusCode.DivByZero) {
        Console.WriteLine($"e did not work {statusE}");
        return (StatusCode.DivByZero, default);    
    }

    return (statusE, e);
}

Console.WriteLine(Sample_StatusCode())

e did not work DivByZero
(DivByZero, 0)


In [ ]:
struct Result<A>
{
    private readonly A value;

    private readonly bool isOk;

    private readonly StatusCode code;

    private Result(A value)
    {
        this.value = value;
    }

    private Result(StatusCode code)
    {
        this.code = code;
    }

    public static Result<A> Identity(A value) => new(value);

    public static Result<A> Error(StatusCode code) => new(code);

    public Result<B> Bind<B>(Func<A, Result<B>> functor)
    {
        if (isOk)
        {
            return functor(value);
        }
        else
        {
            Console.WriteLine($"Value is not set: {code}");
            return Result<B>.Error(code);
        }
    }

    public override string ToString() =>
        isOk
            ? $"Ok({value})"
            : $"Error({code})";
}

static Func<Result<int>> DivBy(int a, int b) =>
    b == 0
        ? Result<int>.Error(StatusCode.DivByZero)
        : Result<int>.Identity(a / b);

static Result<int> Sample_Result(int a, int b)
{
    var cResult = DivBy(a, b);
    var dResult = cResult.Bind(c => DivBy(a, c));
    var eResult = dResult.Bind(d => DivBy(b, d));
    return eResult;
}

Console.WriteLine(Sample_Result(1, 2));